In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Stdlib
from collections import Counter, defaultdict
from datetime import datetime

## Non-std libs
from matplotlib.colors import rgb_to_hsv, to_rgb, hsv_to_rgb
import matplotlib.pyplot as plt

## Local modules
from scn_rrd import config, librenms_meta_utils, librenms_rrd_utils, rrd_utils, plot_utils
from scn_rrd.librenms_api_client import is_device_out_of_service

References
- Unit is millisec.
    - https://github.com/librenms/librenms/blob/master/LibreNMS/Data/Source/FpingResponse.php :: `__construct()`
    - https://github.com/librenms/librenms/blob/fb3c2ecb1c9275108058e76e8af6f145a92235e7/LibreNMS/Polling/ConnectivityHelper.php :: `savePingStats()`

In [ ]:
meta = librenms_meta_utils.read_meta()
meta = meta[ meta['location'].isin(config.PHYS_LOCS) ]
devices = meta[['hostname', 'location']].drop_duplicates()
hostname_to_meta = {
    row['hostname']: row
    for (_, row) in devices.iterrows()
    if not is_device_out_of_service(row['hostname'])
}

logical_loc_to_hostnames = defaultdict(list)
for (hostname, row) in hostname_to_meta.items():
    logical_loc = config.PHYS_LOC_TO_LOGICAL_LOC[row['location']]
    # hack to remove franklin, garfield b/c missing data
    if logical_loc in ['sps-garfield', 'sps-franklin']:
        continue
    logical_loc_to_hostnames[logical_loc].append(hostname)

In [ ]:
hostname_to_df = librenms_rrd_utils.read_rrds(
    hostname_to_meta.keys(),
    librenms_rrd_utils.DEVICE_AGNOSTIC_RRD_FILENAMES.latency,
    '-90days',
)

In [ ]:
logical_locs_ct = len(logical_loc_to_hostnames)
(fig, axez) = plt.subplots(nrows=logical_locs_ct, figsize=(12, 3 * logical_locs_ct), constrained_layout=True)
for (ax, (logical_loc, hostnames)) in zip(axez, logical_loc_to_hostnames.items()):
    for (i, hostname) in enumerate(hostnames):
        df = hostname_to_df[hostname]
        
        # extract the base hue for the location
        color_h = rgb_to_hsv(to_rgb(plot_utils.LOC_TO_COLOR[logical_loc]))[0]
        color_s = (i+1) / len(hostnames)
        
        ax.plot(df['time'], df['ping'], label=hostname, color=hsv_to_rgb((color_h, color_s, 1)))

    ax.set_ylabel('millisec')
    ax.legend()
fig.suptitle('Latency')
None # Hide stdout output of the above line